# Description

This notebook contains examples of CCXT functionality.

## Imports

In [2]:
%load_ext autoreload
%autoreload 2
import logging
import pprint

import ccxt

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import helpers.hsecrets as hsecret

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
display(HTML("""
<style>
/* Jupyter cell is in normal mode when code mirror */
.edit_mode .cell.selected .CodeMirror-focused.cm-fat-cursor {
  /* background-color: #F5F6EB; */
  background-color: rgba(128, 0, 0, 0.1); 
}
/* Jupyter cell is in insert mode when code mirror */
.edit_mode .cell.selected .CodeMirror-focused:not(.cm-fat-cursor) {
  /* background-color: #F6EBF1; */
  background-color: rgba(0, 128, 0, 0.2); 
}
</style>
"""))

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-4dccc445-39d6-441c-a178-0ebbf342cb3f.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='gp_scratch'
  hash='26de62867'
  # Last commits:
    *   26de62867 saggese  Merge branch 'master' into gp_scratch                             (20 minutes ago) Thu Jul 13 08:31:45 2023  (HEAD -> gp_scratch, origin/gp_scratch)
    |\  
    * | 4b839e702 saggese  Update                                                            (  15 hours ago) Wed Jul 12 17:34:03 2023           
    | * 0256f5fca Vlad     CmampTask4631_Create_

In [ ]:
!sudo /bin/bash -c "(source /venv/bin/activate; pip install sagemath)"

In [121]:
from typing import Any, Dict, List

def subset_dict(dict_: Dict, keys: List[Any], *, keep_order: bool = True) -> Dict:
    res = {}
    if keep_order:
        for k, v in dict_.items():
            if k in keys:
                res[k] = v
    else:
        for k in keys:
            res[k] = dict_[k]
    return res

# CCXT

## Resources
- https://github.com/ccxt/ccxt
- https://docs.ccxt.com/#/README
- https://github.com/ccxt/ccxt#readme
- https://ccxt.readthedocs.io/en/latest/index.html
- https://ccxt.readthedocs.io/en/latest/manual.html

## Intro

- CCXT = CryptoCurrency eXchange Trading library

- Connect with cryptocurrency exchanges and trade
- Connect with payment processing services
- Access to market data
- Algorithmic trading
- Strategy backtesting
- Bot programming

- Normalized API for cross-exchange analytics and arbitrage

- There is an async mode using asyncio
  ```
  import ccxt.async_support as ccxt
  ```

- Proxy: in case Cloudflare or your country / IP is rejected
  - Of course an intermediary adds latency
  - Server that acts as an intermediary between the client requesting a resource and the server

## Usage

// https://github.com/ccxt/ccxt#usage

- Public API
  - Unrestricted access to public information for exchanges without account or
    API key
  - E.g.,
    - Market data
    - Order books
    - Price feeds
    - Trade history

- Private API
  - Obtain an API key from exchange website by signing up
    - You might need personal info and verification
  - Manage personal account info
  - Query account balance
  - Query orders
  - Trade
  - Deposit and withdraw fiat and crypto funds

- CCXT supports REST APIs for all exchanges
- CCXT Pro supports WebSocket and FIX

# Exchanges

From https://docs.ccxt.com/#/README?id=exchanges

- Each class implements the public and private API for a particular crypto
  exchange
- The `Exchange` class shares a set of common methods

- Some exchanges offer:
  - Margin trading (i.e., leverage)
  - Derivatives (e.g., futures and options)
  - Dark pools, OTC trading
  - Merchant APIs

- Testnets and mainnets envs
    - Some exchanges offer a separated API for:
      - Testing purposes (aka "sandboxes", "staging environments", "testnets")
      - Trading with real assets (aka "mainnets", "production environments")
    - Typically the sandbox has the same API as the production API but with a
      different URL (and maybe different market)

- Exchange structure
    - Every exchange has properties that can be overridden in the constructor

    - `id`: default id for identification purposes (typically a lower case string)
    - `name`: human-readable exchange name
    - `countries`: where the exchange is operating from
    - `urls`: URLs for private and public APIs, main website, documentation
    - `version`: version of the current exchange API
    - `api`: API endpoints exposed by a crypto exchange
    - `has`: array of exchange capabilities
    - `timeframes`: frequency of available bars (e.g., minutes)
    - `rateLimit`: minimum delay between two consecutive requests
    - `markets`: dictionary of markets indexed by symbols
    - `symbols`: list of symbols available with an exchange
    - `currencies`: array of currencies available

- Exchange metadata
    - Each exchange has a `has` with flags about the exchange capabilities

- Rate limit
    - Exchanges track your user / IP address to throttle querying the API too
      frequently
    - You need to stay under the rate limit to avoid being banned
      - Typical limits are 1 or 2 requests per second
    - CCXT has an experimental rate-limiter that throttles in background
      ```
      exchange.enableRateLimit = True
      ```
      - The state is inside the class instance so one should have a single class

In [11]:
# Print all exchanges.
print(len(ccxt.exchanges), ccxt.exchanges)

117 ['aax', 'alpaca', 'ascendex', 'bequant', 'bibox', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitforex', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitvavo', 'bkex', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcex', 'btcmarkets', 'btctradeua', 'btcturk', 'buda', 'bw', 'bybit', 'bytetrade', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinmate', 'coinone', 'coinspot', 'crex24', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'flowbtc', 'fmfwio', 'gate', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'itbit', 'kraken', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'lbank2', 'liquid', 'luno', 'lykke', 'mercado', 'mexc', 'mexc3', 'ndax', 'nova

In [12]:
# Create Binance exchange.
exchange_id = "binance"
mode = "test"
contract_type = "futures"

# Select credentials for provided exchange.
if mode == "test":
    secrets_id = exchange_id + "_sandbox"
else:
    secrets_id = exchange_id
exchange_params = hsecret.get_secret(secrets_id)

# Enable rate limit.
exchange_params["rateLimit"] = True

# Log into futures/spot market.
if contract_type == "futures":
    exchange_params["options"] = {"defaultType": "future"}

# Create a CCXT Exchange class object.
ccxt_exchange = getattr(ccxt, exchange_id)
print(hprint.to_str("ccxt_exchange"))
exchange = ccxt_exchange(exchange_params)
print(hprint.to_str("exchange"))
if mode == "test":
    exchange.set_sandbox_mode(True)
    _LOG.warning("Running in sandbox mode")
hdbg.dassert(
    exchange.checkRequiredCredentials(),
    msg="Required credentials not passed",
)

ccxt_exchange=<class 'ccxt.binance.binance'>
exchange=ccxt.binance()
WARNING Running in sandbox mode


## Exchange properties

In [13]:
print("exchange=", exchange, type(exchange))

exchange= Binance <class 'ccxt.binance.binance'>


In [14]:
# # Print some properties of the exchange.
# var_names = ["exchange.id", 
#              "exchange.name",
#              "exchange.countries",
#              #"exchange.urls",
#              "exchange.version",
#              "exchange.timeframes",
#              "exchange.timeout",
#              "exchange.rateLimit",
#              "exchange.symbols",
#              "exchange.currencies"]
# for var_name in var_names:
#     print(hprint.to_str(var_name, mode="pprint_color"))

In [15]:
# Name in user-land to identify the exchange.
hprint.pprint_color(exchange.id)

'binance'


In [16]:
# Human readable name.
hprint.pprint_color(exchange.name)

'Binance'


In [17]:
# Which countries the exchange is operating from.
hprint.pprint_color(exchange.countries)

['JP', 'MT']


In [18]:
# Version identifier for exchange API.
hprint.pprint_color(exchange.version)

None


In [19]:
# timeframes for fetchOHLCV().
# TODO(gp): It seems that it has 1s resolution.
hprint.pprint_color(exchange.timeframes)

{'12h': '12h',
 '15m': '15m',
 '1M': '1M',
 '1d': '1d',
 '1h': '1h',
 '1m': '1m',
 '1s': '1s',
 '1w': '1w',
 '2h': '2h',
 '30m': '30m',
 '3d': '3d',
 '3m': '3m',
 '4h': '4h',
 '5m': '5m',
 '6h': '6h',
 '8h': '8h'}


In [21]:
# Binance doesn't have this method.
#hprint.pprint_color(exchange.requiredCredentialsCredentials)

In [22]:
# Exchange decimal precision.
hprint.pprint_color(exchange.precisionMode)

2


In [23]:
hprint.pprint_color(exchange.urls)

{'api': {'dapiPrivate': 'https://testnet.binancefuture.com/dapi/v1',
         'dapiPublic': 'https://testnet.binancefuture.com/dapi/v1',
         'eapiPrivate': 'https://testnet.binanceops.com/eapi/v1',
         'eapiPublic': 'https://testnet.binanceops.com/eapi/v1',
         'fapiPrivate': 'https://testnet.binancefuture.com/fapi/v1',
         'fapiPrivateV2': 'https://testnet.binancefuture.com/fapi/v2',
         'fapiPublic': 'https://testnet.binancefuture.com/fapi/v1',
         'private': 'https://testnet.binance.vision/api/v3',
         'public': 'https://testnet.binance.vision/api/v3',
         'v1': 'https://testnet.binance.vision/api/v1'},
 'apiBackup': {'dapiData': 'https://dapi.binance.com/futures/data',
               'dapiPrivate': 'https://dapi.binance.com/dapi/v1',
               'dapiPrivateV2': 'https://dapi.binance.com/dapi/v2',
               'dapiPublic': 'https://dapi.binance.com/dapi/v1',
               'eapiPrivate': 'https://eapi.binance.com/eapi/v1',
             

In [24]:
hprint.pprint_color(exchange.api)

{'dapiData': {'get': {'basis': 1,
                      'globalLongShortAccountRatio': 1,
                      'openInterestHist': 1,
                      'takerBuySellVol': 1,
                      'topLongShortAccountRatio': 1,
                      'topLongShortPositionRatio': 1}},
 'dapiPrivate': {'delete': {'allOpenOrders': 1,
                            'batchOrders': 5,
                            'listenKey': 1,
                            'order': 1},
                 'get': {'account': 5,
                         'adlQuantile': 5,
                         'allOrders': {'cost': 20, 'noSymbol': 40},
                         'balance': 1,
                         'forceOrders': {'cost': 20, 'noSymbol': 50},
                         'income': 20,
                         'leverageBracket': 1,
                         'openOrder': 1,
                         'openOrders': {'cost': 1, 'noSymbol': 5},
                         'order': 1,
                         'orderAmendment': 

In [25]:
exchange.loadMarkets()
print(hprint.list_to_str(exchange.markets.keys(), tag="market_list"))

market_list: (227) BTC/USDT ETH/USDT BCH/USDT XRP/USDT EOS/USDT LTC/USDT TRX/USDT ETC/USDT LINK/USDT XLM/USDT ADA/USDT DASH/USDT ZEC/USDT XTZ/USDT BNB/USDT ATOM/USDT ONT/USDT IOTA/USDT BAT/USDT VET/USDT NEOUSDT QTUM/USDT IOST/USDT THETA/USDT ALGO/USDT ZIL/USDT KNC/USDT ZRX/USDT COMP/USDT OMG/USDT DOGE/USDT SXP/USDT KAVA/USDT BAND/USDT RLC/USDT WAVES/USDT MKR/USDT SNX/USDT DOT/USDT DEFI/USDT YFI/USDT BAL/USDT CRV/USDT TRBUSDT RUNE/USDT SUSHI/USDT SRMUSDT EGLD/USDT SOL/USDT ICXUSDT STORJ/USDT UNI/USDT AVAX/USDT FTM/USDT HNT/USDT ENJ/USDT FLM/USDT TOMO/USDT REN/USDT KSM/USDT NEAR/USDT AAVE/USDT FIL/USDT RSRUSDT LRC/USDT MATIC/USDT OCEAN/USDT BELUSDT CTKUSDT AXS/USDT ALPHAUSDT ZEN/USDT SKL/USDT GRT/USDT BNTUSDT 1INCH/USDT UNFI/USDT BTC/BUSD CHZ/USDT SAND/USDT ANKR/USDT LIT/USDT REEF/USDT RVNUSDT SFPUSDT XEMUSDT BTCSTUSDT COTI/USDT CHR/USDT MANA/USDT ALICE/USDT HBARUSDT ONE/USDT LINA/USDT STMX/USDT DENT/USDT CELR/USDT HOT/USDT MTL/USDT OGN/USDT NKNUSDT DGBUSDT 1000SHIB/USDT BAKEUSDT GTCUSDT

In [26]:
print(hprint.list_to_str(exchange.currencies, tag="currencies"))

currencies: (197) 1000FLOKI 1000LUNC 1000PEPE 1000SHIB 1000XEC 1INCH AAVE ACH ADA AGIX ALGO ALICE ALPHA AMB ANC ANKR ANT APE API3 APT AR ARB ARPA ASTR ATA ATOM AUCTION AUDIO AVAX AXS BAKE BAL BAND BAT BCH BEL BLUEBIRD BLUR BNB BNT BNX BTC BTCDOM BTCST BUSD C98 CELO CELR CFX CHR CHZ CKB COMBO COMP COTI CRV CTK CTSI CVX DAR DASH DEFI DENT DGB DOGE DOT DUSK DYDX EDU EGLD ENJ ENS EOS ETC ETH FET FIL FLM FLOW FOOTBALL FTM FXS GAL GALA GMT GMX GRT GTC HBAR HFT HIGH HNT HOOK HOT ICP ICP2 ICX ID IDEX IMX INJ IOST IOTA IOTX JASMY JOE KAVA KEY KLAY KNC KSM LDO LEVER LINA LINK LIT LPT LQTY LRC LTC LUNA2 MAGIC MANA MASK MATIC MAV MDT MINA MKR MTL NEAR NEO NKN NMR OCEAN OGN OMG ONE ONT OP PEOPLE PERP PHB QNT QTUM RAD RDNT REEF REN RLC RNDR ROSE RSR RUNE RVN SAND SFP SKL SNX SOL SPELL SRM SSV STG STMX STORJ STX SUI SUSHI SXP T THETA TOMO TRB TRU TRX UMA UNFI UNI USDC USDT VET WAVE WAVES WOO XEM XLM XMR XRP XTZ XVG XVS YFI ZEC ZEN ZIL ZRX



In [27]:
hprint.pprint_color(exchange.commonCurrencies)

{'BCC': 'BCC', 'BCHABC': 'BCH', 'BCHSV': 'BSV', 'XBT': 'BTC', 'YOYO': 'YOYOW'}


## Exchange metadata

In [28]:
# Flags for exchange capabilities (true, false, emulated).
# CORS = cross-origin resource sharing.
hprint.pprint_color(exchange.has)

{'CORS': None,
 'addMargin': True,
 'borrowMargin': True,
 'cancelAllOrders': True,
 'cancelOrder': True,
 'cancelOrders': None,
 'createDepositAddress': False,
 'createLimitOrder': True,
 'createMarketOrder': True,
 'createOrder': True,
 'createPostOnlyOrder': None,
 'createReduceOnlyOrder': True,
 'createStopLimitOrder': True,
 'createStopMarketOrder': False,
 'createStopOrder': True,
 'editOrder': 'emulated',
 'fetchAccounts': None,
 'fetchBalance': True,
 'fetchBidsAsks': True,
 'fetchBorrowInterest': True,
 'fetchBorrowRate': True,
 'fetchBorrowRateHistories': False,
 'fetchBorrowRateHistory': True,
 'fetchBorrowRates': False,
 'fetchBorrowRatesPerSymbol': False,
 'fetchCanceledOrders': False,
 'fetchClosedOrder': False,
 'fetchClosedOrders': 'emulated',
 'fetchCurrencies': True,
 'fetchDeposit': False,
 'fetchDepositAddress': True,
 'fetchDepositAddresses': False,
 'fetchDepositAddressesByNetwork': False,
 'fetchDeposits': True,
 'fetchFundingFee': None,
 'fetchFundingFees': None

## Rate limit

# Markets

- Valuables
    - Valuables are exchanged at each market
      - E.g.,
        - instruments
        - symbols
        - assets
        - trading pairs
        - currencies
        - tokens
        - contract

- Exchange and Market
    - Every Exchange offers multiple Markets
    - A Market is usually a pair of currencies (e.g., crypto, fiat)

## Currency structure

- Each currency has an associated dictionary
    - `id`: currency id within the exchange
    - `code`: `cctx` representation of the currency
    - `name`: human readable currency name
    - `fee`: withdrawal fee
    - `active`: indicates whether trading and funding this currency is possible
    - `info`: dictionary of non-common market properties
    - `precision`
    - `limits`: min and max for withdrawals

## Market structure

- `id` string representing the instrument within the exchange (e.g., `btcusd`)
- `baseId` (e.g., `btc`), `quoteId` (e.g., `usd`) are exchange-specific ids
- `symbol` string code representing the trading pair
  - E.g., typically `BaseCurrency/QuoteCurrency` (e.g., `BTC/USD`)
  - This is standard in `ccxt`
- `base` (e.g., `BTC`) / `quote` (`USD`) standardized currency code
- `active`: indicates whether trading this market is possible
  - The cache of the markets should be refreshed periodically
- `maker`: maker fees paid when you provide liquidity to the exchange
  - E.g., you make an order and someone else fills it
  - A negative fee means a rebate
- `taker`: taker fees paid when you take liquidity from the exchange (i.e., you
  fill someone else's order)
- `tierBased`: whether the fee depends on your trading tier (e.g., amount of
  trading done over a period of time)
- `info`: non-common market properties
- `precision`: precision used for price, amount, and cost
  - E.g., decimal places, significant digits, tick size

In [29]:
# A market is an associative array.

#market_id = exchange.markets_by_id["1000FLOKIUSDT"]
market_id = exchange.markets_by_id["ETHUSDT"]
print(type(market_id))
hprint.pprint_color(market_id)

<class 'dict'>
{'active': True,
 'base': 'ETH',
 'baseId': 'ETH',
 'contract': True,
 'contractSize': 1.0,
 'delivery': False,
 'expiry': None,
 'expiryDatetime': None,
 'future': True,
 'id': 'ETHUSDT',
 'info': {'baseAsset': 'ETH',
          'baseAssetPrecision': '8',
          'contractType': 'PERPETUAL',
          'deliveryDate': '4133404800000',
          'filters': [{'filterType': 'PRICE_FILTER',
                       'maxPrice': '95000.02',
                       'minPrice': '18.62',
                       'tickSize': '0.01'},
                      {'filterType': 'LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MARKET_LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
                      {'

In [32]:
# Trade instrument within the exchange. This is the internal representation of each exchange.
market = exchange.markets["ETH/USDT"]
print(market["id"])

ETHUSDT


In [33]:
# Trade instrument in CCXT user-land (unified). Typically referrend as "base/quote".
print(market["symbol"])

ETH/USDT


In [34]:
# Market ids (unified)
print(market["base"], market["quote"])

ETH USDT


In [35]:
# Symbol ids (not unified).
print(market["baseId"], market["quoteId"])

ETH USDT


In [36]:
print(hprint.to_str('market["active"] market["maker"] market["taker"] market["percentage"]'))

market["active"]=True, market["maker"]=0.0002, market["taker"]=0.0004, market["percentage"]=True


In [37]:
# Market-specific properties.
hprint.pprint_color(market["info"])

{'baseAsset': 'ETH',
 'baseAssetPrecision': '8',
 'contractType': 'PERPETUAL',
 'deliveryDate': '4133404800000',
 'filters': [{'filterType': 'PRICE_FILTER',
              'maxPrice': '95000.02',
              'minPrice': '18.62',
              'tickSize': '0.01'},
             {'filterType': 'LOT_SIZE',
              'maxQty': '10000',
              'minQty': '0.001',
              'stepSize': '0.001'},
             {'filterType': 'MARKET_LOT_SIZE',
              'maxQty': '10000',
              'minQty': '0.001',
              'stepSize': '0.001'},
             {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
             {'filterType': 'MAX_NUM_ALGO_ORDERS', 'limit': '10'},
             {'filterType': 'MIN_NOTIONAL', 'notional': '5.0'},
             {'filterType': 'PERCENT_PRICE',
              'multiplierDecimal': '4',
              'multiplierDown': '0.9000',
              'multiplierUp': '1.1000'}],
 'liquidationFee': '0.020000',
 'maintMarginPercent': '2.5000',
 'marginAsset': '

## Network structure

## Precision and limits

In [38]:
# limits = min, max for prices / amounts (aka volumes) / costs (= price * amount)
# precision = precision for prices / amounts / costs accepted in order values when placing orders
# They are not related.

hprint.pprint_color(market["limits"], tag="limits", sep="\n")
hprint.pprint_color(market["precision"], tag="precision", sep="\n")

limits= 
{'amount': {'max': 10000.0, 'min': 0.001},
 'cost': {'max': None, 'min': 5.0},
 'leverage': {'max': None, 'min': None},
 'market': {'max': 10000.0, 'min': 0.001},
 'price': {'max': 95000.02, 'min': 18.62}}
precision= 
{'amount': 3, 'base': 8, 'price': 2, 'quote': 8}


In [39]:
# Min / max amount (i.e., volume) for an order.
print(market["limits"]["amount"])

{'min': 0.001, 'max': 10000.0}


In [40]:
# How many decimal digits.

In [41]:
# Each exchange has their own way of rounding and truncating.

In [42]:
exchange.precisionMode

2

In [43]:
ccxt.TICK_SIZE

4

In [44]:
ccxt.SIGNIFICANT_DIGITS

3

In [45]:
ccxt.DECIMAL_PLACES

2

In [46]:
# From https://docs.ccxt.com/#/README?id=formatting-to-precision
#ccxt.base.decimal_to_precision.amount_to_precision(symbol, amount)

## Loading markets

In [47]:
markets = exchange.load_markets()

## Symbols and Market Ids

### Symbols and market ids

* Currency code
- = a code of 3 to 5 uppercase letters
- E.g., `BTC`, `ETH`, `USD`, `XRP`

* Symbol
- = a pair of currencies separated by a slash
  - E.g., `BTC/USD`
- The first currency is called the "base currency"
- The second currency is called the "quote currency"
  - BASE / QUOTE

* Market Ids
- Market ids are unique per exchange and are used in REST request-response
  - E.g., the same BTC/USD pair can be called in different ways on different
    markets (e.g., `BTCUSD`, `btc/usd`)
- `CCTX` abstracts market ids into standardized symbols

* Market symbol vs market ids
- "Market symbols" are the abstract representation
- "Market ids" are specific of each market

### Methods for markets and currencies

// notebook

### Naming consistency

* Products
- Some exchanges call markets as "pairs" or "products"
- `CCXT` considers each exchange as having one or more "trading markets"
  - Each market has an `id` and a `symbol`
  - Most symbols are typically a currency pair

* Exchange -> Markets -> Symbols -> Currencies
- The logic is:
  - Exchange (name of the exchange, e.g., Binance)
  - Markets (a "product" that is traded, e.g., the pair `BTC/USD`)
  - Symbols (a pair of traded currencies separated by slash)
  - Currencies (the currency code, e.g., `BTC` and `USD`)

- The same currency:
  - can have different names on different exchanges
  - has changed name over time (e.g., `XBT` -> `BTC`, `USD` = `USDT`)

* Expiring / perpetual futures
- Aka "swaps"
- Futures market symbol have:
  - Underlying currency
  - Quoting currency
  - Settlement currency
  - Identifier for a settlement date (typically as YYMMDD)

- E.g., `BTC/USDT:BTC-211225`
  - BTC/USDT futures contract settled in BTC (inverse) on 2021-12-25
- E.g., `BTC/USDT:USDT-211225`
  - BTC/USDT futures contract settled in USDT (linear, vanilla)

* Perpetual futures
- Aka "perpetual swaps"
- E.g., `BTC/USDT:BTC`

In [48]:
exchange.load_markets();

In [101]:
# Get the market structure.
market = exchange.markets["ETH/USDT"]
hprint.pprint_color(market)

{'active': True,
 'base': 'ETH',
 'baseId': 'ETH',
 'contract': True,
 'contractSize': 1.0,
 'delivery': False,
 'expiry': None,
 'expiryDatetime': None,
 'feeSide': 'get',
 'future': True,
 'id': 'ETHUSDT',
 'info': {'baseAsset': 'ETH',
          'baseAssetPrecision': '8',
          'contractType': 'PERPETUAL',
          'deliveryDate': '4133404800000',
          'filters': [{'filterType': 'PRICE_FILTER',
                       'maxPrice': '306177',
                       'minPrice': '39.86',
                       'tickSize': '0.01'},
                      {'filterType': 'LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MARKET_LOT_SIZE',
                       'maxQty': '2000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
                      {

In [102]:
# Print a subset of interesting values for Market structure.
var_names = [
    "id",
    "symbol",
    "base",
    "quote",
    "baseId",
    "quoteId",
    "active",
    "maker",
    "taker",
    "tierBased",
    "info",
    "precision",
    "limits",
]
# for var_name in var_names:
#     print(f"--> {var_name}=", hprint.pprint_pformat(market[var_name]))
hprint.pprint_color(
    subset_dict(market, var_names))

{'active': True,
 'base': 'ETH',
 'baseId': 'ETH',
 'id': 'ETHUSDT',
 'info': {'baseAsset': 'ETH',
          'baseAssetPrecision': '8',
          'contractType': 'PERPETUAL',
          'deliveryDate': '4133404800000',
          'filters': [{'filterType': 'PRICE_FILTER',
                       'maxPrice': '306177',
                       'minPrice': '39.86',
                       'tickSize': '0.01'},
                      {'filterType': 'LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MARKET_LOT_SIZE',
                       'maxQty': '2000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
                      {'filterType': 'MAX_NUM_ALGO_ORDERS', 'limit': '10'},
                      {'filterType': 'MIN_NOTIONAL', 'notional': '5.0'},
             

In [51]:
# Print all the symbols in one exchange.
symbols = exchange.symbols
print(symbols)

['1000FLOKIUSDT', '1000LUNC/BUSD', '1000LUNC/USDT', '1000PEPE/USDT', '1000SHIB/BUSD', '1000SHIB/USDT', '1000XEC/USDT', '1INCH/USDT', 'AAVE/USDT', 'ACH/USDT', 'ADA/BUSD', 'ADA/USDT', 'AGIX/BUSD', 'AGIX/USDT', 'ALGO/USDT', 'ALICE/USDT', 'ALPHAUSDT', 'AMB', 'AMB/BUSD', 'AMB/USDT', 'ANC/BUSD', 'ANKR/USDT', 'ANT/USDT', 'APE/USDT', 'API3/USDT', 'APT/BUSD', 'APT/USDT', 'AR/USDT', 'ARB/BUSD', 'ARB/USDT', 'ARPA/USDT', 'ASTR/USDT', 'ATA/USDT', 'ATOM/USDT', 'AUCTION/BUSD', 'AUDIO/USDT', 'AVAX/USDT', 'AXS/USDT', 'BAKEUSDT', 'BAL/USDT', 'BAND/USDT', 'BAT/USDT', 'BCH/USDT', 'BELUSDT', 'BLUEBIRD/USDT', 'BLUR/USDT', 'BNB/BUSD', 'BNB/USDT', 'BNTUSDT', 'BNX/USDT', 'BTC/BUSD', 'BTC/USDT', 'BTCDOM/USDT', 'BTCSTUSDT', 'BTCUSDT_230929', 'C98/USDT', 'CELO/USDT', 'CELR/USDT', 'CFX/USDT', 'CHR/USDT', 'CHZ/USDT', 'CKB/USDT', 'COMBO/USDT', 'COMP/USDT', 'COTI/USDT', 'CRV/USDT', 'CTKUSDT', 'CTSIUSDT', 'CVX/USDT', 'DAR/USDT', 'DASH/USDT', 'DEFI/USDT', 'DENT/USDT', 'DGBUSDT', 'DOGE/USDT', 'DOGEBUSD', 'DOT/BUSD', 'DO

In [52]:
# Print a dictionary of all currencies.
currencies = exchange.currencies
hprint.pprint_color(currencies)

{'1000FLOKI': {'code': '1000FLOKI',
               'id': '1000FLOKI',
               'numericId': None,
               'precision': 8},
 '1000LUNC': {'code': '1000LUNC',
              'id': '1000LUNC',
              'numericId': None,
              'precision': 8},
 '1000PEPE': {'code': '1000PEPE',
              'id': '1000PEPE',
              'numericId': None,
              'precision': 8},
 '1000SHIB': {'code': '1000SHIB',
              'id': '1000SHIB',
              'numericId': None,
              'precision': 8},
 '1000XEC': {'code': '1000XEC',
             'id': '1000XEC',
             'numericId': None,
             'precision': 8},
 '1INCH': {'code': '1INCH', 'id': '1INCH', 'numericId': None, 'precision': 8},
 'AAVE': {'code': 'AAVE', 'id': 'AAVE', 'numericId': None, 'precision': 8},
 'ACH': {'code': 'ACH', 'id': 'ACH', 'numericId': None, 'precision': 8},
 'ADA': {'code': 'ADA', 'id': 'ADA', 'numericId': None, 'precision': 8},
 'AGIX': {'code': 'AGIX', 'id': 'AGIX', 'numericI

In [53]:
#market_id = exchange.markets_by_id["1000FLOKIUSDT"]
market_id = exchange.markets_by_id["ETHUSDT"]
print(type(market_id))
hprint.pprint_color(market_id)

<class 'dict'>
{'active': True,
 'base': 'ETH',
 'baseId': 'ETH',
 'contract': True,
 'contractSize': 1.0,
 'delivery': False,
 'expiry': None,
 'expiryDatetime': None,
 'future': True,
 'id': 'ETHUSDT',
 'info': {'baseAsset': 'ETH',
          'baseAssetPrecision': '8',
          'contractType': 'PERPETUAL',
          'deliveryDate': '4133404800000',
          'filters': [{'filterType': 'PRICE_FILTER',
                       'maxPrice': '95000.02',
                       'minPrice': '18.62',
                       'tickSize': '0.01'},
                      {'filterType': 'LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MARKET_LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
                      {'

In [54]:
market_id["symbol"]

'ETH/USDT'

## Market cache force reload

# Implicit API methods

## API methods / endpoints

- API methods / endpoints
    - Each exchange offers a set of API methods (aka "endpoints") that are HTTP URLs
      for querying various types of information
    - All endpoints return JSON responses

- E.g., an endpoint for:
  - getting a list of markets from an exchange
  - retrieving an order book
  - retrieving trade history
  - cancelling orders
  
- Endpoints are defined in `api` property of an exchange

## Implicit API methods

- In practice each API method is mapped on callable Python function
- Each function can be called with a dictionary of parameters and return an
  unparsed JSON from the exchange API
- The method is available in both camelCase and under_score notation

## Public / private / unified

- Each `Exchange` implements:
  - a public / private API for all endpoints
  - a unified API supporting a subset of common methods

- One should:
  - use unified methods
  - use the private method as fallback

## Public / Private API

- Public API doesn't require authentication
    - Aka "market data", "basic api", "market api", "mapi"
    - E.g.,
        - Allow to access market data
        - Price feeds
        - Order books
        - Trade history
        - Bars

- Private API requires authentication
    - Aka "trading api", "tapi"
    - E.g.,
        - Manage personal account info
        - Query account balances
        - Trade
        - Create deposit
        - Request withdrawal
        - Query orders

- Some exchanges also expose a "merchant API" to accept crypto and fiat as payments
    - Aka "merchant", "wallet", "ecapi" (for e-commerce)

* Synch vs async calls
- `CCXT` supports asyncio
- The same methods are available but decorated with `asyncio` keyword

* Returned objects
- Public and private APIs return raw JSON objects (representing the response from
  the exchange)
- Unified APIs return a JSON object in a common format across all exchanges

In [55]:
# Print a list of all the methods in an exchange.
print(dir(ccxt.binance()))

['__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'account', 'accounts', 'addMargin', 'add_margin', 'aggregate', 'aiohttpProxy', 'aiohttpTrustEnv', 'aiohttp_proxy', 'aiohttp_trust_env', 'alias', 'amountToPrecision', 'amount_to_precision', 'api', 'apiKey', 'arrayConcat', 'array_concat', 'asyncioLoop', 'asyncio_loop', 'balance', 'base16ToBinary', 'base16_to_binary', 'base58Alphabet', 'base58Decoder', 'base58Encoder', 'base58ToBinary', 'base58_alphabet', 'base58_decoder', 'base58_encoder', 'base58_to_binary', 'base64ToBinary', 'base64ToString', 'base64_to_binary', 'base64_to_string', 'base64urlencode', 'baseCurrencies', 'base_currencies', 'binaryConcat', 'binaryConcatArray', 'binaryToBase

## Synchronous vs asynchronous

- CCXT supports async concurrency mode with async/await
- Use `asyncio` and `aiohttp`
- The methods are the same but they are decorated with `async`

```
import asyncio
import ccx.async_support as ccxt

...
```

## API parameters

- Public / private API endpoints differ from exchange to exchange
    - Most methods accept an array of key-value params
- Return a raw JSON object

- Unified API return JSON in a common format uniform across all exchanges

# Unified API

- fetch...
  - Markets
  - Currencies
  - OrderBook
  - Status
  - Trades
  - Ticker
  - Balance
- create...
  - Order
  - LimitBuyOrder / LimitSellOrder
  - MarketBuyOrder / MarketSellOrder
  - CancelOrder
- fetch orders
  - Open
  - Canceled
  - Closed
- fetch
    - my trades
    - open interest
    - transactions
    - deposit
    - withdrawals

- A `param` argument is a dictionary of exchange-specific params you want override

## Pagination
- Most exchange APIs return a certain number of the most recent objects
- You can't get all the objects in one call
  - You need to paginate, i.e., fetch portions of data one by one
  - Pagination can be performed based on id, time, or page number

# Public API

https://docs.ccxt.com/#/README?id=public-api

In [56]:
exchange.fetchMarkets()

[{'id': 'BTCUSDT',
  'lowercaseId': 'btcusdt',
  'symbol': 'BTC/USDT',
  'base': 'BTC',
  'quote': 'USDT',
  'settle': 'USDT',
  'baseId': 'BTC',
  'quoteId': 'USDT',
  'settleId': 'USDT',
  'type': 'future',
  'spot': False,
  'margin': False,
  'swap': True,
  'future': True,
  'delivery': False,
  'option': False,
  'active': True,
  'contract': True,
  'linear': True,
  'inverse': False,
  'taker': 0.0004,
  'maker': 0.0002,
  'contractSize': 1.0,
  'expiry': None,
  'expiryDatetime': None,
  'strike': None,
  'optionType': None,
  'precision': {'amount': 3, 'price': 1, 'base': 8, 'quote': 8},
  'limits': {'leverage': {'min': None, 'max': None},
   'amount': {'min': 0.001, 'max': 1000.0},
   'price': {'min': 302.1, 'max': 936574.4},
   'cost': {'min': 5.0, 'max': None},
   'market': {'min': 0.001, 'max': 1000.0}},
  'info': {'symbol': 'BTCUSDT',
   'pair': 'BTCUSDT',
   'contractType': 'PERPETUAL',
   'deliveryDate': '4133404802000',
   'onboardDate': '1569398400000',
   'status': 

In [57]:
exchange.fetchCurrencies()

In [58]:
# Not supported for Binance
# exchange.fetchStatus()

In [98]:
symbol = "BTC/USDT"
data = exchange.fetchOrderBook(symbol)
#data = exchange.fetchL2OrderBook(symbol)
print("keys=", data.keys())

key_names = ["symbol", "timestamp", "datetime", "nonce", "bids"]
hprint.pprint_color(
    subset_dict(data, key_names))

keys= dict_keys(['symbol', 'bids', 'asks', 'timestamp', 'datetime', 'nonce'])
# symbol
'BTC/USDT'
# timestamp
1689243005611
# datetime
'2023-07-13T10:10:05.611Z'
# nonce
3054457027505
# bids
[[30545.4, 41.911],
 [30545.3, 5.795],
 [30545.2, 0.021],
 [30545.1, 1.487],
 [30545.0, 0.004],
 [30544.9, 0.023],
 [30544.8, 0.001],
 [30544.7, 0.05],
 [30544.6, 0.164],
 [30544.5, 0.35],
 [30544.4, 0.015],
 [30544.3, 0.001],
 [30544.2, 0.008],
 [30544.1, 0.351],
 [30544.0, 0.28],
 [30543.9, 0.083],
 [30543.8, 0.14],
 [30543.7, 0.015],
 [30543.6, 0.041],
 [30543.5, 0.012],
 [30543.4, 0.037],
 [30543.3, 0.096],
 [30543.2, 0.155],
 [30543.1, 0.053],
 [30543.0, 0.162],
 [30542.9, 4.959],
 [30542.8, 0.001],
 [30542.7, 0.601],
 [30542.6, 0.011],
 [30542.5, 0.991],
 [30542.4, 0.214],
 [30542.3, 0.051],
 [30542.2, 0.882],
 [30542.1, 0.165],
 [30542.0, 2.449],
 [30541.9, 1.998],
 [30541.8, 0.854],
 [30541.7, 0.067],
 [30541.6, 1.71],
 [30541.5, 0.192],
 [30541.4, 0.997],
 [30541.2, 0.021],
 [30541.1, 0.32

In [60]:
data = exchange.fetchTrades(symbol)
hprint.pprint_color(data[:2])

[{'amount': 0.1,
  'cost': 3016.24,
  'datetime': '2023-07-10T09:20:21.381Z',
  'fee': None,
  'fees': [],
  'id': '152760771',
  'info': {'T': '1688980821381',
           'a': '152760771',
           'f': '263456596',
           'l': '263456596',
           'm': True,
           'p': '30162.40',
           'q': '0.100'},
  'order': None,
  'price': 30162.4,
  'side': 'sell',
  'symbol': 'BTC/USDT',
  'takerOrMaker': 'taker',
  'timestamp': 1688980821381,
  'type': None},
 {'amount': 5.103,
  'cost': 153918.7272,
  'datetime': '2023-07-10T09:20:22.509Z',
  'fee': None,
  'fees': [],
  'id': '152760772',
  'info': {'T': '1688980822509',
           'a': '152760772',
           'f': '263456597',
           'l': '263456597',
           'm': True,
           'p': '30162.40',
           'q': '5.103'},
  'order': None,
  'price': 30162.4,
  'side': 'sell',
  'symbol': 'BTC/USDT',
  'takerOrMaker': 'taker',
  'timestamp': 1688980822509,
  'type': None}]


In [61]:
symbol = "BTC/USDT"
data = exchange.fetchTicker(symbol)
hprint.pprint_color(data)

{'ask': None,
 'askVolume': None,
 'average': 30272.85,
 'baseVolume': 102545.049,
 'bid': None,
 'bidVolume': None,
 'change': 54.1,
 'close': 30299.9,
 'datetime': '2023-07-10T09:40:54.188Z',
 'high': 30479.0,
 'info': {'closeTime': '1688982054188',
          'count': '103078',
          'firstId': '263354172',
          'highPrice': '30479.00',
          'lastId': '263457255',
          'lastPrice': '30299.90',
          'lastQty': '0.001',
          'lowPrice': '29941.50',
          'openPrice': '30245.80',
          'openTime': '1688895600000',
          'priceChange': '54.10',
          'priceChangePercent': '0.179',
          'quoteVolume': '3105934785.56',
          'symbol': 'BTCUSDT',
          'volume': '102545.049',
          'weightedAvgPrice': '30288.49'},
 'last': 30299.9,
 'low': 29941.5,
 'open': 30245.8,
 'percentage': 0.179,
 'previousClose': None,
 'quoteVolume': 3105934785.56,
 'symbol': 'BTC/USDT',
 'timestamp': 1688982054188,
 'vwap': 30288.49}


## FetchBalance

In [62]:
symbol = "BTC/USDT"
data = exchange.fetchBalance()
hprint.pprint_color(data)

{'BNB': {'free': 8.8774531, 'total': 2.45657689, 'used': 0.0},
 'BTC': {'free': 0.06883463, 'total': 0.0, 'used': 0.0},
 'BUSD': {'free': 2135.53177178, 'total': 1000.20004, 'used': 0.0},
 'ETH': {'free': 1.1146111, 'total': 0.0, 'used': 0.0},
 'USDC': {'free': 2135.07405103, 'total': 0.0, 'used': 0.0},
 'USDT': {'free': 2177.5013884, 'total': 4601.17963671, 'used': 3981.28813729},
 'datetime': None,
 'free': {'BNB': 8.8774531,
          'BTC': 0.06883463,
          'BUSD': 2135.53177178,
          'ETH': 1.1146111,
          'USDC': 2135.07405103,
          'USDT': 2177.5013884},
 'info': {'assets': [{'asset': 'BTC',
                      'availableBalance': '0.06883463',
                      'crossUnPnl': '0.00000000',
                      'crossWalletBalance': '0.00000000',
                      'initialMargin': '0.00000000',
                      'maintMargin': '0.00000000',
                      'marginAvailable': True,
                      'marginBalance': '0.00000000',
      

- L1: market price only
- L2: order volume aggregated by price
- L3: each order is kept separated

## Market price

In [63]:
#symbol = exchange.symbols[0]
symbol = "BTC/USDT"
print(symbol)
orderbook = exchange.fetch_order_book(symbol)
bid = orderbook["bids"][0][0] if len(orderbook["bids"]) > 0 else None
ask = orderbook["asks"][0][0] if len(orderbook["asks"]) > 0 else None
spread = (ask - bid) if (bid and ask) else None
print(exchange.id, {"bid": bid, "ask": ask, "spread": spread})

BTC/USDT
binance {'bid': 30134.0, 'ask': 30275.1, 'spread': 141.09999999999854}


## FetchTicker()

In [64]:
symbol = "BTC/USDT"
data = exchange.fetchTicker(symbol)
hprint.pprint_color(data)

{'ask': None,
 'askVolume': None,
 'average': 30204.45,
 'baseVolume': 102529.415,
 'bid': None,
 'bidVolume': None,
 'change': -140.9,
 'close': 30134.0,
 'datetime': '2023-07-10T09:41:03.201Z',
 'high': 30479.0,
 'info': {'closeTime': '1688982063201',
          'count': '103023',
          'firstId': '263354229',
          'highPrice': '30479.00',
          'lastId': '263457257',
          'lastPrice': '30134.00',
          'lastQty': '0.219',
          'lowPrice': '29941.50',
          'openPrice': '30274.90',
          'openTime': '1688895660000',
          'priceChange': '-140.90',
          'priceChangePercent': '-0.465',
          'quoteVolume': '3105461266.15',
          'symbol': 'BTCUSDT',
          'volume': '102529.415',
          'weightedAvgPrice': '30288.49'},
 'last': 30134.0,
 'low': 29941.5,
 'open': 30274.9,
 'percentage': -0.465,
 'previousClose': None,
 'quoteVolume': 3105461266.15,
 'symbol': 'BTC/USDT',
 'timestamp': 1688982063201,
 'vwap': 30288.49}


## OHLCV bars

In [65]:
symbol = "BTC/USDT"
data = exchange.fetchOHLCV(symbol)
# O, H, L, C, V
hprint.pprint_color(data[:5])

[[1688952120000, 30090.9, 30282.0, 30090.0, 30282.0, 48.604],
 [1688952180000, 30282.0, 30282.0, 30090.0, 30120.0, 61.407],
 [1688952240000, 30093.0, 30282.0, 30090.0, 30090.0, 96.284],
 [1688952300000, 30090.0, 30282.0, 30090.0, 30200.0, 115.67],
 [1688952360000, 30090.9, 30282.0, 30090.0, 30090.0, 47.743]]


- The info from the current candle may be incomplete until the candle is closed

- Exchanges provide
    - (fast) primary data (e.g., order books, trades, fills)
        - WebSockets might be faster than REST API
    - (slow) secondary data calculated from primary data (e.g., OHLCV bars)
        - It might be faster to compute data locally

## Public trades

In [66]:
symbol = "BTC/USDT"
data = exchange.fetch_trades(symbol)
hprint.pprint_color(data[:2])

[{'amount': 0.375,
  'cost': 11362.4625,
  'datetime': '2023-07-10T09:20:26.438Z',
  'fee': None,
  'fees': [],
  'id': '152760775',
  'info': {'T': '1688980826438',
           'a': '152760775',
           'f': '263456600',
           'l': '263456600',
           'm': False,
           'p': '30299.90',
           'q': '0.375'},
  'order': None,
  'price': 30299.9,
  'side': 'buy',
  'symbol': 'BTC/USDT',
  'takerOrMaker': 'taker',
  'timestamp': 1688980826438,
  'type': None},
 {'amount': 0.211,
  'cost': 6364.2664,
  'datetime': '2023-07-10T09:20:28.179Z',
  'fee': None,
  'fees': [],
  'id': '152760776',
  'info': {'T': '1688980828179',
           'a': '152760776',
           'f': '263456601',
           'l': '263456601',
           'm': True,
           'p': '30162.40',
           'q': '0.211'},
  'order': None,
  'price': 30162.4,
  'side': 'sell',
  'symbol': 'BTC/USDT',
  'takerOrMaker': 'taker',
  'timestamp': 1688980828179,
  'type': None}]


## Borrow rates

- When short trading or trading with leverage on a spot market, currency must be
  borrowed

In [67]:
# Binance doesn't support this.
#exchange.fetchBorrowRatesPerSymbol(symbol)

## Leverage tiers

In [68]:
symbol = "BTC/USDT"
data =  exchange.fetchMarketLeverageTiers(symbol)
hprint.pprint_color(data[0])
hprint.pprint_color(data[-1])

{'currency': 'USDT',
 'info': {'bracket': '1',
          'cum': '0.0',
          'initialLeverage': '125',
          'maintMarginRatio': '0.004',
          'notionalCap': '50000',
          'notionalFloor': '0'},
 'maintenanceMarginRate': 0.004,
 'maxLeverage': 125.0,
 'maxNotional': 50000.0,
 'minNotional': 0.0,
 'tier': 1.0}
{'currency': 'USDT',
 'info': {'bracket': '10',
          'cum': '1.014213E8',
          'initialLeverage': '1',
          'maintMarginRatio': '0.5',
          'notionalCap': '500000000',
          'notionalFloor': '300000000'},
 'maintenanceMarginRate': 0.5,
 'maxLeverage': 1.0,
 'maxNotional': 500000000.0,
 'minNotional': 300000000.0,
 'tier': 10.0}


## Funding rate

In [69]:
data = exchange.fetchFundingRate(symbol)
hprint.pprint_color(data)

{'datetime': '2023-07-10T09:41:17.000Z',
 'estimatedSettlePrice': 30102.08968791,
 'fundingDatetime': '2023-07-10T16:00:00.000Z',
 'fundingRate': 0.00015788,
 'fundingTimestamp': 1689004800000,
 'indexPrice': 30131.55186813,
 'info': {'estimatedSettlePrice': '30102.08968791',
          'indexPrice': '30131.55186813',
          'interestRate': '0.00010000',
          'lastFundingRate': '0.00015788',
          'markPrice': '30135.01004989',
          'nextFundingTime': '1689004800000',
          'symbol': 'BTCUSDT',
          'time': '1688982077000'},
 'interestRate': 0.0001,
 'markPrice': 30135.01004989,
 'nextFundingDatetime': None,
 'nextFundingRate': None,
 'nextFundingTimestamp': None,
 'previousFundingDatetime': None,
 'previousFundingRate': None,
 'previousFundingTimestamp': None,
 'symbol': 'BTC/USDT',
 'timestamp': 1688982077000}


In [70]:
data = exchange.fetchFundingRateHistory(symbol)
hprint.pprint_color(data[:3])

[{'datetime': '2023-06-07T08:00:00.000Z',
  'fundingRate': 0.0001,
  'info': {'fundingRate': '0.00010000',
           'fundingTime': '1686124800000',
           'symbol': 'BTCUSDT'},
  'symbol': 'BTC/USDT',
  'timestamp': 1686124800000},
 {'datetime': '2023-06-07T16:00:00.000Z',
  'fundingRate': 0.003,
  'info': {'fundingRate': '0.00300000',
           'fundingTime': '1686153600000',
           'symbol': 'BTCUSDT'},
  'symbol': 'BTC/USDT',
  'timestamp': 1686153600000},
 {'datetime': '2023-06-08T00:00:00.000Z',
  'fundingRate': 0.003,
  'info': {'fundingRate': '0.00300000',
           'fundingTime': '1686182400000',
           'symbol': 'BTCUSDT'},
  'symbol': 'BTC/USDT',
  'timestamp': 1686182400000}]


## Open interest

In [71]:
# Binance doesn't support this.
# data = exchange.fetchOpenInterest(symbol)
# hprint.pprint_color(data[:3])

# Private API

https://docs.ccxt.com/#/README?id=private-api

- `fetchBalance`
- `createOrder`, `cancelOrder`
- `fetchOrder`, `fetchOpenOrder`, `fetchCanceledOrder`, `fetchClosedOrder`
- `fetchMyTrades`
- `fetchPositions`
- `fetchTransactions`
- `fetchLedger`

## Authentication
- Handled automatically if API key provided
- Generate nonce (integer and increasing, e.g., 32-bit Unix Timestamp in seconds)
- Append public API key and nonce to the endpoint params, serialize, sign
- Append signature to HTTP headers

- `apiKey`
    - non-secret
    - sent over HTTPS
- `secret`
    - private key
    - used to sign requests locally
    - used together with the nonce
    - signature sent with public key to authenticate
- `uid`
    - some exchanges generate a user id
- `password`
    - some exchanges use also password for trading

In [4]:
import oms.hsecrets.secret_identifier as ohsseide

exchange_id = "binance"
#account_type = "sandbox"
account_type = "trading"
stage = "preprod"
#secret_id = "4"
secret_id = 4

secret_identifier = ohsseide.SecretIdentifier(
    exchange_id, stage, account_type, secret_id
    )
print(secret_identifier)

# Prepare exchange params.
exchange_params = hsecret.get_secret(str(secret_identifier))
#print(exchange_params)
exchange_params["rateLimit"] = False
exchange_params["options"] = {"defaultType": "future"}

# Build the exchange object.
ccxt_exchange = getattr(ccxt, exchange_id)
exchange = ccxt_exchange(exchange_params)

exchange.options["adjustForTimeDifference"] = True

/app/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


binance.preprod.trading.4


In [95]:
# Check what type of authentication an exchange needs.
hprint.pprint_color(exchange.requiredCredentials)

{'apiKey': True,
 'login': False,
 'password': False,
 'privateKey': False,
 'secret': True,
 'token': False,
 'twofa': False,
 'uid': False,
 'walletAddress': False}


In [7]:
# Check that the credentials work.
# It throws an `AuthenticationError` if login fails.
exchange.check_required_credentials()

True

## Overriding the nonce

## Accounts

* `fetchAccounts()`
- Return accounts and sub-accounts in a dict.

In [31]:
# Binance doesn't have.
# exchange.fetchAccounts()

## Account balance

* `fetchBalance()`
- Query for balance and get the amount of funds available

In [8]:
balance = exchange.fetchBalance()

In [9]:
hprint.pprint_color(balance)

{'ADA': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'BNB': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'BTC': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'BUSD': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'DOT': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'ETH': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'TUSD': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'USDC': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'USDP': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'USDT': {'free': 716.27859396, 'total': 741.50646893, 'used': 25.12769435},
 'XRP': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'datetime': None,
 'free': {'ADA': 0.0,
          'BNB': 0.0,
          'BTC': 0.0,
          'BUSD': 0.0,
          'DOT': 0.0,
          'ETH': 0.0,
          'TUSD': 0.0,
          'USDC': 0.0,
          'USDP': 0.0,
          'USDT': 716.27859396,
          'XRP': 0.0},
 'info': {'assets': [{'asset': 'BTC',
                      'availableBalance': '0.00000000',
                      'crossUnPnl': '0.00000000',

In [10]:
balance.keys()

dict_keys(['info', 'BTC', 'ADA', 'XRP', 'TUSD', 'DOT', 'BNB', 'ETH', 'USDT', 'USDP', 'USDC', 'BUSD', 'timestamp', 'datetime', 'free', 'used', 'total'])

In [94]:
#print(hprint.to_str('balance["timestamp"] balance["datetime"]'))
key_names = ["timestamp", "datetime"]
hprint.pprint_color(
    subset_dict(balance, key_names))

{'datetime': None, 'timestamp': None}


In [12]:
# Coins available for trading.
hprint.pprint_color(balance["free"])

{'ADA': 0.0,
 'BNB': 0.0,
 'BTC': 0.0,
 'BUSD': 0.0,
 'DOT': 0.0,
 'ETH': 0.0,
 'TUSD': 0.0,
 'USDC': 0.0,
 'USDP': 0.0,
 'USDT': 716.27859396,
 'XRP': 0.0}


In [13]:
# Coins on hold / locked.
hprint.pprint_color(balance["used"])

{'ADA': 0.0,
 'BNB': 0.0,
 'BTC': 0.0,
 'BUSD': 0.0,
 'DOT': 0.0,
 'ETH': 0.0,
 'TUSD': 0.0,
 'USDC': 0.0,
 'USDP': 0.0,
 'USDT': 25.12769435,
 'XRP': 0.0}


In [14]:
# Total coins (=free + used).
hprint.pprint_color(balance["total"])

{'ADA': 0.0,
 'BNB': 0.0,
 'BTC': 0.0,
 'BUSD': 0.0,
 'DOT': 0.0,
 'ETH': 0.0,
 'TUSD': 0.0,
 'USDC': 0.0,
 'USDP': 0.0,
 'USDT': 741.50646893,
 'XRP': 0.0}


In [15]:
# Indexed by coins.
hprint.pprint_color(balance["BTC"])

{'free': 0.0, 'total': 0.0, 'used': 0.0}


In [45]:
# `info` contains the response (unparsed) from the exchange.
hprint.pprint_color(balance["info"])

{'assets': [{'asset': 'BTC',
             'availableBalance': '0.00000000',
             'crossUnPnl': '0.00000000',
             'crossWalletBalance': '0.00000000',
             'initialMargin': '0.00000000',
             'maintMargin': '0.00000000',
             'marginAvailable': True,
             'marginBalance': '0.00000000',
             'maxWithdrawAmount': '0.00000000',
             'openOrderInitialMargin': '0.00000000',
             'positionInitialMargin': '0.00000000',
             'unrealizedProfit': '0.00000000',
             'updateTime': '0',
             'walletBalance': '0.00000000'},
            {'asset': 'ADA',
             'availableBalance': '0.00000000',
             'crossUnPnl': '0.00000000',
             'crossWalletBalance': '0.00000000',
             'initialMargin': '0.00000000',
             'maintMargin': '0.00000000',
             'marginAvailable': True,
             'marginBalance': '0.00000000',
             'maxWithdrawAmount': '0.00000000',
       

In [46]:
balance["info"].keys()

dict_keys(['feeTier', 'canTrade', 'canDeposit', 'canWithdraw', 'updateTime', 'multiAssetsMargin', 'totalInitialMargin', 'totalMaintMargin', 'totalWalletBalance', 'totalUnrealizedProfit', 'totalMarginBalance', 'totalPositionInitialMargin', 'totalOpenOrderInitialMargin', 'totalCrossWalletBalance', 'totalCrossUnPnl', 'availableBalance', 'maxWithdrawAmount', 'assets', 'positions'])

In [19]:
# Print info about one asset.
print(len(balance["info"]["assets"]))
hprint.pprint_color(balance["info"]["assets"][0])

11
{'asset': 'BTC',
 'availableBalance': '0.00000000',
 'crossUnPnl': '0.00000000',
 'crossWalletBalance': '0.00000000',
 'initialMargin': '0.00000000',
 'maintMargin': '0.00000000',
 'marginAvailable': True,
 'marginBalance': '0.00000000',
 'maxWithdrawAmount': '0.00000000',
 'openOrderInitialMargin': '0.00000000',
 'positionInitialMargin': '0.00000000',
 'unrealizedProfit': '0.00000000',
 'updateTime': '0',
 'walletBalance': '0.00000000'}


In [21]:
# Print info about one position.
print(len(balance["info"]["positions"]))
hprint.pprint_color(balance["info"]["positions"][0])

239
{'askNotional': '0',
 'bidNotional': '0',
 'entryPrice': '0.0',
 'initialMargin': '0',
 'isolated': False,
 'isolatedWallet': '0',
 'leverage': '20',
 'maintMargin': '0',
 'maxNotional': '25000',
 'notional': '0',
 'openOrderInitialMargin': '0',
 'positionAmt': '0',
 'positionInitialMargin': '0',
 'positionSide': 'BOTH',
 'symbol': 'SUSHIUSDT',
 'unrealizedProfit': '0.00000000',
 'updateTime': '0'}


In [16]:
balance["info"]["feeTier"]

'0'

In [91]:
# for key in ["canTrade", "canDeposit", "canWithdraw"]:
#     print(hprint.to_str(f'balance["info"]["{key}"]'))
hprint.pprint_color(
    subset_dict(
        balance["info"],
        ["canTrade", "canDeposit", "canWithdraw"]))

{'canDeposit': True, 'canTrade': True, 'canWithdraw': True}


In [53]:
balance["info"]["updateTime"]

'0'

In [ ]:
# https://www.binance.com/en/support/faq/leverage-and-margin-of-usd%E2%93%A2-m-futures-360033162192
# - USD-M futures: margin and settlement in USDT (Tether) and BUSD (Binance Stable coin)
# - COIN-M futures: margin and settlement in alt-coins

In [92]:
# for key in ['totalInitialMargin',
#             'totalMaintMargin',
#             'totalWalletBalance', 
#             'totalUnrealizedProfit',
#             'totalMarginBalance',
#             'totalPositionInitialMargin',
#             'totalOpenOrderInitialMargin',
#             'totalCrossWalletBalance',
#             'totalCrossUnPnl', 
#             'availableBalance', 'maxWithdrawAmount']:
#     print(hprint.to_str(f'balance["info"]["{key}"]'))

key_names = ['totalInitialMargin',
             'totalMaintMargin',
             'totalWalletBalance', 
             'totalUnrealizedProfit',
             'totalMarginBalance',
             'totalPositionInitialMargin',
             'totalOpenOrderInitialMargin',
             'totalCrossWalletBalance',
             'totalCrossUnPnl', 
             'availableBalance',
             'maxWithdrawAmount']
hprint.pprint_color(
    subset_dict(balance["info"], key_names))

{'availableBalance': '716.27859396',
 'maxWithdrawAmount': '716.27859396',
 'totalCrossUnPnl': '0.38660012',
 'totalCrossWalletBalance': '741.11986881',
 'totalInitialMargin': '25.12769435',
 'totalMaintMargin': '9.43887851',
 'totalMarginBalance': '741.50646893',
 'totalOpenOrderInitialMargin': '0.00000000',
 'totalPositionInitialMargin': '25.12769435',
 'totalUnrealizedProfit': '0.38660012',
 'totalWalletBalance': '741.11986881'}


## Orders

https://docs.ccxt.com/#/README?id=orders

- You can query orders by an id or symbol
- Some exchanges might not have all methods

In [32]:
# for k, v in exchange.has.items():
#     if "order" in k or "Order" in k:
#         print(k, v)

hprint.pprint_color(subset_dict(
    exchange.has, [
    "fetchOrder", "fetchOrders",
    "fetchOpenOrder", "fetchOpenOrders",
    "fetchClosedOrder", "fetchClosedOrders"]))

{'fetchClosedOrder': False,
 'fetchClosedOrders': 'emulated',
 'fetchOpenOrder': False,
 'fetchOpenOrders': True,
 'fetchOrder': True,
 'fetchOrders': True}


### Understanding the Orders API design

- `fetch{,Open,Canceled}Orders()`
- `fetchMyTrades()`: history of settled trades
- `createOrder()`
- `cancelOrder()`

- All methods returning a list of trades / orders, accept a `since` and `limit` arg
    - Without `since` the method returns the default set of results from the exchange (e.g., last 24 hours or last N trades
    - Some exchanges provide pagination through the `params` arg

In [134]:
orders = exchange.fetchOrders("BTC/USDT", limit=2)

In [135]:
hprint.pprint_color(orders[0])

{'info': {'orderId': '170783916148',
          'symbol': 'BTCUSDT',
          'status': 'CANCELED',
          'clientOrderId': 'x-xcKtGhcu48111d25c4a7838729d6f2',
          'price': '30893.50',
          'avgPrice': '0.00000',
          'origQty': '0.006',
          'executedQty': '0',
          'cumQuote': '0',
          'timeInForce': 'GTC',
          'type': 'LIMIT',
          'reduceOnly': False,
          'closePosition': False,
          'side': 'BUY',
          'positionSide': 'BOTH',
          'stopPrice': '0',
          'workingType': 'CONTRACT_PRICE',
          'priceProtect': False,
          'origType': 'LIMIT',
          'time': '1689265527208',
          'updateTime': '1689265599480'},
 'id': '170783916148',
 'clientOrderId': 'x-xcKtGhcu48111d25c4a7838729d6f2',
 'timestamp': 1689265527208,
 'datetime': '2023-07-13T16:25:27.208Z',
 'lastTradeTimestamp': None,
 'symbol': 'BTC/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'reduceOnly': False,
 'side': 

- closed orders are not trades (aka fills)
- an order doesn't have `fee`
- trades have `fee` and `cost`

In [132]:
closed_orders = exchange.fetchClosedOrders("BTC/USDT", limit=2)

In [133]:
hprint.pprint_color(closed_orders[0])

{'info': {'orderId': '170786184487',
          'symbol': 'BTCUSDT',
          'status': 'FILLED',
          'clientOrderId': 'x-xcKtGhcubbf5e270ce85157b29444b',
          'price': '30897.50',
          'avgPrice': '30899.60000',
          'origQty': '0.004',
          'executedQty': '0.004',
          'cumQuote': '123.59840',
          'timeInForce': 'GTC',
          'type': 'LIMIT',
          'reduceOnly': False,
          'closePosition': False,
          'side': 'SELL',
          'positionSide': 'BOTH',
          'stopPrice': '0',
          'workingType': 'CONTRACT_PRICE',
          'priceProtect': False,
          'origType': 'LIMIT',
          'time': '1689265828679',
          'updateTime': '1689265828679'},
 'id': '170786184487',
 'clientOrderId': 'x-xcKtGhcubbf5e270ce85157b29444b',
 'timestamp': 1689265828679,
 'datetime': '2023-07-13T16:30:28.679Z',
 'lastTradeTimestamp': None,
 'symbol': 'BTC/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'reduceOnly': F

### Order structure

https://docs.ccxt.com/#/README?id=order-structure

In [136]:
order = orders[0]

In [89]:
order.keys()

dict_keys(['info', 'id', 'clientOrderId', 'timestamp', 'datetime', 'lastTradeTimestamp', 'symbol', 'type', 'timeInForce', 'postOnly', 'reduceOnly', 'side', 'price', 'stopPrice', 'amount', 'cost', 'average', 'filled', 'remaining', 'status', 'fee', 'trades', 'fees'])

In [137]:
var_names = ["id",
             # You can tag the order.
             "clientOrderId",
             "timestamp",
             "datetime",
             "lastTradeTimestamp"]
hprint.pprint_color(
    subset_dict(order, var_names, keep_order=False))

{'id': '170783916148',
 'clientOrderId': 'x-xcKtGhcu48111d25c4a7838729d6f2',
 'timestamp': 1689265527208,
 'datetime': '2023-07-13T16:25:27.208Z',
 'lastTradeTimestamp': None}


In [138]:
var_names = [
    # 
    'status',
    'symbol',
    # E.g., market, limit
    'type',
    'timeInForce',
    'side',
    # Price in quote currency.
    'price',
    # Average filling price.
    'average',
    # How much is ordered vs filled vs remaining.
    'amount',
    'filled',
    'remaining',
    # = filled * price
    'cost',
    'fee',
    'fees',
    # List of trades.
    'trades',
    #
    'stopPrice',
    'postOnly',
    'reduceOnly',
]
hprint.pprint_color(subset_dict(order, var_names, keep_order=False))

{'status': 'canceled',
 'symbol': 'BTC/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'side': 'buy',
 'price': 30893.5,
 'average': None,
 'amount': 0.006,
 'filled': 0.0,
 'remaining': 0.006,
 'cost': 0.0,
 'fee': None,
 'fees': [],
 'trades': [],
 'stopPrice': None,
 'postOnly': False,
 'reduceOnly': False}


In [127]:
hprint.pprint_color(order["info"])

{'orderId': '170643031299',
 'symbol': 'BTCUSDT',
 'status': 'CANCELED',
 'clientOrderId': 'x-xcKtGhcu6e473c588377f85f69952b',
 'price': '30400.10',
 'avgPrice': '0.00000',
 'origQty': '0.001',
 'executedQty': '0',
 'cumQuote': '0',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'reduceOnly': False,
 'closePosition': False,
 'side': 'SELL',
 'positionSide': 'BOTH',
 'stopPrice': '0',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': False,
 'origType': 'LIMIT',
 'time': '1689237027940',
 'updateTime': '1689237099073'}


### Placing orders

https://docs.ccxt.com/#/README?id=placing-orders

- Limit orders
    - Amount in base currency (how much you want to buy / sell)
    - Price in quote currency (for which price you want to buy / sell)
- Trigger orders
    - Wait for a condition on a market (trigger) and then a market order is placed
- Stop loss orders / Take profit orders
    - Like a special trigger order
    - When price passes a certain value, a market / limit order is triggered

- `createOrder` is used to place orders:
  - symbol
  - side
      - buy `BTC/USD`, receive quote currency (BTC) for base currency (USD)
      - sell `BTC/USD`: receive USD for BTC
  - type
      - market
      - limit
  - amount
      - Typically expressed in terms of the base currency
      - For some exchanges it is dependent on the side of the order
  - price
      - in units of the quotes currency
  - params
      - params specific to the exchange API
  - a successful order returns an order structure
 
 
- Limit orders
    - `create_limit_order`
    - `create_buy_limit_order`
    - `create_sell_limit_order`
    - They are placed on the exchange for a certain price
    - They are fullfilled (closed) when:
        - there are no orders at a better price
        - a market / limit order for a price that matches or exceeds the price of the limit order

- Market orders
    - Executed immediately using orders from the top of the other side of the book
      (i.e., orders are chosen with best price available)
    - You are not guaranteed that the order is executed for the price you observe
      prior to placing the order because
        - network latency
        - high loads on the exchange
        - price volatility
        - order walking the book

### Editing orders

https://docs.ccxt.com/#/README?id=editing-orders

An order can also be edited

TODO(gp): Unclear what happens. I believe it's cancelled and placed again

### Canceling orders

https://docs.ccxt.com/#/README?id=canceling-orders

- `cancelOrder`: cancel a single order
- `candelOrders`: cancel multiple orders
- `cancelAllOrders`: cancel all the open orders

It is possible that an order gets executed while the cancel command is being executed
    - Then a `NetworkError` or `OrderNotFound` are thrown

## My trades

https://docs.ccxt.com/#/README?id=my-trades

### How orders are related to trades
- A trade is also called a "fill"
- Each trade is a result of order execution
- One order may result in several trades
    - i.e., an order can be filled with one or more trades
    - one-to-many relationship

- Each trade is a result of an order execution (matching opposing orders)
- An execution of one order can result in several trades (i.e., filled with
  multiple trades)

### Personal trades
- Typically exchanges use pagination to return all the trades

### Trade structure

https://docs.ccxt.com/#/README?id=trade-structure

In [141]:
order_id = None
trades = exchange.fetch_my_trades(symbol, limit=2)

In [143]:
trade = trades[0]

In [144]:
hprint.pprint_color(trade)

{'info': {'symbol': 'BTCUSDT',
          'id': '3932745371',
          'orderId': '170669758249',
          'side': 'BUY',
          'price': '30536.90',
          'qty': '0.004',
          'realizedPnl': '0',
          'marginAsset': 'USDT',
          'quoteQty': '122.14760',
          'commission': '0.02442952',
          'commissionAsset': 'USDT',
          'time': '1689243340610',
          'positionSide': 'BOTH',
          'buyer': True,
          'maker': True},
 'timestamp': 1689243340610,
 'datetime': '2023-07-13T10:15:40.610Z',
 'symbol': 'BTC/USDT',
 'id': '3932745371',
 'order': '170669758249',
 'type': None,
 'side': 'buy',
 'takerOrMaker': 'maker',
 'price': 30536.9,
 'amount': 0.004,
 'cost': 122.1476,
 'fee': {'cost': 0.02442952, 'currency': 'USDT'},
 'fees': [{'currency': 'USDT', 'cost': 0.02442952}]}


In [145]:
trade.keys()

dict_keys(['info', 'timestamp', 'datetime', 'symbol', 'id', 'order', 'type', 'side', 'takerOrMaker', 'price', 'amount', 'cost', 'fee', 'fees'])

In [147]:
var_names = ["id",
             "timestamp",
             "datetime"]
hprint.pprint_color(
    subset_dict(trade, var_names, keep_order=False))

{'id': '3932745371',
 'timestamp': 1689243340610,
 'datetime': '2023-07-13T10:15:40.610Z'}


In [149]:
var_names = ["symbol",
             "order",
             "type",
             "side",
             "takerOrMaker",
             # Price in quote currency.
             "price",
             # Amount of base currency.
             "amount",
             # Total cost, i.e., price * amount.
             "cost",
             # Fees.
             "fee",
             "fees"]
hprint.pprint_color(
    subset_dict(trade, var_names, keep_order=False))

{'symbol': 'BTC/USDT',
 'order': '170669758249',
 'type': None,
 'side': 'buy',
 'takerOrMaker': 'maker',
 'price': 30536.9,
 'amount': 0.004,
 'cost': 122.1476,
 'fee': {'cost': 0.02442952, 'currency': 'USDT'},
 'fees': [{'currency': 'USDT', 'cost': 0.02442952}]}


In [148]:
# Original decoded JSON from the exchange.
var_names = ["info"]
hprint.pprint_color(
    subset_dict(trade, var_names, keep_order=False))

{'info': {'symbol': 'BTCUSDT',
          'id': '3932745371',
          'orderId': '170669758249',
          'side': 'BUY',
          'price': '30536.90',
          'qty': '0.004',
          'realizedPnl': '0',
          'marginAsset': 'USDT',
          'quoteQty': '122.14760',
          'commission': '0.02442952',
          'commissionAsset': 'USDT',
          'time': '1689243340610',
          'positionSide': 'BOTH',
          'buyer': True,
          'maker': True}}


### Trades by Order id

In [152]:
# Not supported by Binance.
# order_id = "170643031299"
# trades = exchange.fetch_order_trades(order_id, symbol, limit=2)

## Ledger

- = history of changes / actions done by the user affecting the balance

- funding (deposits / withdrawals)
- profits / losses from trades
- trading fees
- rebates, etc.

In [153]:
# Not supported by Binance.
# exchange.fetchLedger()

AttributeError: 'binance' object has no attribute 'fetchLedger'

### Ledger entry structure

https://docs.ccxt.com/#/README?id=ledger-entry-structure

## Deposit

https://docs.ccxt.com/#/README?id=deposit

In [155]:
exchange.fetchDeposits()

[{'info': {'id': '3444986436108417024',
   'amount': '1000',
   'coin': 'USDT',
   'network': 'ETH',
   'status': '1',
   'address': '0x2a17ad951e8d85a24c8330d5db8377daf42e03e3',
   'addressTag': '',
   'txId': '0x4886c564a18afe1f91c00f3c5085f3de0c6a9cbdabbb5138cbea32b892813cae',
   'insertTime': '1683295593000',
   'transferType': '0',
   'confirmTimes': '64/6',
   'unlockConfirm': '64',
   'walletType': '0'},
  'id': '3444986436108417024',
  'txid': '0x4886c564a18afe1f91c00f3c5085f3de0c6a9cbdabbb5138cbea32b892813cae',
  'timestamp': 1683295593000,
  'datetime': '2023-05-05T14:06:33.000Z',
  'network': 'ETH',
  'address': '0x2a17ad951e8d85a24c8330d5db8377daf42e03e3',
  'addressTo': '0x2a17ad951e8d85a24c8330d5db8377daf42e03e3',
  'addressFrom': None,
  'tag': None,
  'tagTo': None,
  'tagFrom': None,
  'type': 'deposit',
  'amount': 1000.0,
  'currency': 'USDT',
  'status': 'ok',
  'updated': None,
  'internal': False,
  'fee': None},
 {'info': {'id': '3532118562884369152',
   'amount'

### Withdraw

### Transactions

* Trading fees
- = amount paid to the exchange
- Typically it is a percentage of volume traded

* Funding fees
- Fees for depositing and withdrawing
- Crypto transaction fees


In [ ]:
balance = exchange.fetchBalance()

balance

In [ ]:
balance.keys()

# CCXT Pro

// https://ccxt.readthedocs.io/en/latest/ccxt.pro.manual.html

- Standard CCXT uses request-response based APIs (e.g., REST)
- CCXT Pro uses connection-based streaming API (e.g., WebSocket)

- `fetch*` methods are replaced with `watch*` methods

- CCXT Pro manages the connection-based interface transparently to the user
  - On the first call to a `watch*` method a connection is established
  - If the connection already exists, it is reused
  - The library watches the status of the connection and keeps it alive

* Sub interface
- Allows to subscribe to a stream of data and lister for it
  - E.g., streaming public market data (e.g., order book, bars)

* Pub interface
- Allows to send data requests towards the server
  - E.g., placing / cancelling orders

* Incremental data structures
- In many cases, the application listening needs to
  - keep a local snapshot of the data in memory
  - merge the updates received from the exchange server (aka "deltas")

- CCXT Pro automatically handles this